In [3]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

# Import requests library
import requests

# Import the API key
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?"

In [4]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [8]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each lat and long combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # if the city name is unique, add it to the cities list
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count
len(cities)

592

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+") + "&APPID=" + weather_api_key
    
    # Log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | shingu
Processing Record 2 of Set 1 | dilla
Processing Record 3 of Set 1 | rikitea
Processing Record 4 of Set 1 | souillac
Processing Record 5 of Set 1 | kodiak
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | bathsheba
Processing Record 8 of Set 1 | carnarvon
Processing Record 9 of Set 1 | yellowknife
Processing Record 10 of Set 1 | punta arenas
Processing Record 11 of Set 1 | port elizabeth
Processing Record 12 of Set 1 | asau
Processing Record 13 of Set 1 | jamestown
Processing Record 14 of Set 1 | usinsk
Processing Record 15 of Set 1 | coihaique
Processing Record 16 of Set 1 | coahuayana
Processing Record 17 of Set 1 | rawson
Processing Record 18 of Set 1 | marquette
Processing Record 19 of Set 1 | khatanga
Processing Record 20 of Set 1 | torbay
Processing Record 21 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 22 of Set 1 | dudinka
Processing Record 

Processing Record 42 of Set 4 | linjiang
Processing Record 43 of Set 4 | houma
Processing Record 44 of Set 4 | dunedin
Processing Record 45 of Set 4 | yurya
City not found. Skipping...
Processing Record 46 of Set 4 | ondjiva
Processing Record 47 of Set 4 | villazon
Processing Record 48 of Set 4 | tanjong karang
City not found. Skipping...
Processing Record 49 of Set 4 | petropavlovsk-kamchatskiy
Processing Record 50 of Set 4 | nanortalik
Processing Record 1 of Set 5 | vaini
Processing Record 2 of Set 5 | thompson
Processing Record 3 of Set 5 | lavrentiya
Processing Record 4 of Set 5 | gambo
Processing Record 5 of Set 5 | mecca
Processing Record 6 of Set 5 | pontalina
Processing Record 7 of Set 5 | lolua
City not found. Skipping...
Processing Record 8 of Set 5 | sentyabrskiy
City not found. Skipping...
Processing Record 9 of Set 5 | chegdomyn
Processing Record 10 of Set 5 | watertown
Processing Record 11 of Set 5 | do rud
City not found. Skipping...
Processing Record 12 of Set 5 | dhema

Processing Record 28 of Set 8 | samana
Processing Record 29 of Set 8 | kainantu
Processing Record 30 of Set 8 | oktyabrskiy
Processing Record 31 of Set 8 | samalaeulu
City not found. Skipping...
Processing Record 32 of Set 8 | dniprorudne
Processing Record 33 of Set 8 | mazagao
Processing Record 34 of Set 8 | boende
Processing Record 35 of Set 8 | constantine
Processing Record 36 of Set 8 | ayia marina
Processing Record 37 of Set 8 | hofn
Processing Record 38 of Set 8 | ha giang
Processing Record 39 of Set 8 | yantal
Processing Record 40 of Set 8 | tongren
Processing Record 41 of Set 8 | wagar
Processing Record 42 of Set 8 | plouzane
Processing Record 43 of Set 8 | comodoro rivadavia
Processing Record 44 of Set 8 | tooele
Processing Record 45 of Set 8 | sofiysk
City not found. Skipping...
Processing Record 46 of Set 8 | ceyhan
Processing Record 47 of Set 8 | eyl
Processing Record 48 of Set 8 | chagda
City not found. Skipping...
Processing Record 49 of Set 8 | pran buri
Processing Recor

Processing Record 11 of Set 12 | mahajanga
Processing Record 12 of Set 12 | broken hill
Processing Record 13 of Set 12 | okha
Processing Record 14 of Set 12 | morondava
Processing Record 15 of Set 12 | northport
Processing Record 16 of Set 12 | gouyave
Processing Record 17 of Set 12 | pauini
Processing Record 18 of Set 12 | poum
Processing Record 19 of Set 12 | ouango
Processing Record 20 of Set 12 | ichinoseki
Processing Record 21 of Set 12 | jahanabad
Processing Record 22 of Set 12 | hailar
Processing Record 23 of Set 12 | gotsu
Processing Record 24 of Set 12 | cheuskiny
City not found. Skipping...
Processing Record 25 of Set 12 | tucuman
Processing Record 26 of Set 12 | pshekhskaya
Processing Record 27 of Set 12 | mayo
Processing Record 28 of Set 12 | angoche
Processing Record 29 of Set 12 | maniitsoq
Processing Record 30 of Set 12 | kisangani
Processing Record 31 of Set 12 | ocampo
Processing Record 32 of Set 12 | tigzirt
Processing Record 33 of Set 12 | burnie
Processing Record 34

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Shingu,33.7333,135.9833,290.88,45,0,0.35,JP,2022-11-09 01:04:11
1,Dilla,6.4100,38.3100,289.71,78,100,1.49,ET,2022-11-09 01:07:16
2,Rikitea,-23.1203,-134.9692,297.47,87,100,6.37,PF,2022-11-09 01:06:36
3,Souillac,-20.5167,57.5167,295.04,79,90,4.00,MU,2022-11-09 01:07:16
4,Kodiak,57.7900,-152.4072,278.88,87,100,10.29,US,2022-11-09 01:07:17
5,Ushuaia,-54.8000,-68.3000,281.96,53,75,1.54,AR,2022-11-09 01:07:17
6,Bathsheba,13.2167,-59.5167,301.49,83,40,9.77,BB,2022-11-09 01:07:11
7,Carnarvon,-24.8667,113.6333,294.11,54,59,4.27,AU,2022-11-09 01:06:59
8,Yellowknife,62.4560,-114.3525,263.55,79,100,4.59,CA,2022-11-09 01:07:15
9,Punta Arenas,-53.1500,-70.9167,284.21,58,75,13.38,CL,2022-11-09 01:06:46


In [15]:
# Re-order the columns
new_column_order = ['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed']
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Shingu,JP,2022-11-09 01:04:11,33.7333,135.9833,290.88,45,0,0.35
1,Dilla,ET,2022-11-09 01:07:16,6.4100,38.3100,289.71,78,100,1.49
2,Rikitea,PF,2022-11-09 01:06:36,-23.1203,-134.9692,297.47,87,100,6.37
3,Souillac,MU,2022-11-09 01:07:16,-20.5167,57.5167,295.04,79,90,4.00
4,Kodiak,US,2022-11-09 01:07:17,57.7900,-152.4072,278.88,87,100,10.29
5,Ushuaia,AR,2022-11-09 01:07:17,-54.8000,-68.3000,281.96,53,75,1.54
6,Bathsheba,BB,2022-11-09 01:07:11,13.2167,-59.5167,301.49,83,40,9.77
7,Carnarvon,AU,2022-11-09 01:06:59,-24.8667,113.6333,294.11,54,59,4.27
8,Yellowknife,CA,2022-11-09 01:07:15,62.4560,-114.3525,263.55,79,100,4.59
9,Punta Arenas,CL,2022-11-09 01:06:46,-53.1500,-70.9167,284.21,58,75,13.38


In [16]:
# Create an output file to save the DataFrame in as a CSV in a new folder for that file

# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")